In [ ]:
import torch
idens = torch.load("/mnt/rd/identity_df.pt", map_location="cpu")
clip_embs = torch.load("/home/ubuntu/AutoLoRADiscovery/celeba_clip_arc_embeddings.pt", map_location="cpu")

In [ ]:
idens.columns 
# ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
#        'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
#        'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
#        'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
#        'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
#        'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
#        'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
#        'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
#        'Wearing_Necklace', 'Wearing_Necktie', 'Young', 'identity']

In [ ]:
## create dataset for training

index_titles= ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young']



class CelebaClipDataset(torch.utils.data.Dataset):
    def __init__(self, idens, clip_embs, is_train=False):
        self.clip_embs = clip_embs
        self.idens = idens
        n_idens = []
        for i, index in enumerate(idens.index.values):
            if index in clip_embs:
                n_idens.append((i, index))
        available_idens = n_idens
        if is_train:
            available_idens = available_idens[:int(len(available_idens) * 0.8)]
        else:
            available_idens = available_idens[int(len(available_idens) * 0.8):]
        self.available_idens = available_idens
        
    def __len__(self):
        return len(self.available_idens)
    def __getitem__(self, idx):
        idx, _idx = self.available_idens[idx]
        return self.clip_embs[_idx][1], self.idens.iloc[idx].values[:-1].astype(float)

In [ ]:
train_dataset = CelebaClipDataset(idens, clip_embs, is_train=True)
test_dataset = CelebaClipDataset(idens, clip_embs, is_train=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)

print("train dataset size", len(train_dataset))
print("test dataset size", len(test_dataset))

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
class CLiPToFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            # nn.LayerNorm(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            # nn.LayerNorm(256),
            nn.Dropout(0.2),
            nn.Linear(256, 40),
            nn.ReLU(),
            # nn.LayerNorm(40),
            nn.Linear(40, 40),

            # nn.Tanh()
        )
    def forward(self, x):
        return self.seq(x)

In [ ]:
model = CLiPToFeatures().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
# train-test loop , with binary cross entropy loss
for epoch in range(1000):
    model.train()
    train_loss = 0
    for clip, iden in train_loader:
        clip = clip.cuda().float()
        # normalize clip
        clip = clip / clip.norm(dim=-1, keepdim=True)
        iden = iden.cuda().float() # is from -1 to 1, so we need to normalize it
        iden = (iden + 1) / 2
        optimizer.zero_grad()
        out = model(clip)
        loss = F.binary_cross_entropy_with_logits(out, iden)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    model.eval()
    with torch.no_grad():
        total_loss = 0
        total_correct = 0

        for clip, iden in test_loader:
            clip = clip.cuda().float()
            # normalize clip
            clip = clip / clip.norm(dim=-1, keepdim=True)
            iden = iden.cuda().float() # is from -1 to 1, so we need to normalize it
            iden = (iden + 1) / 2
            out = model(clip)
            loss = F.binary_cross_entropy_with_logits(out, iden)
            total_loss += loss.item()

            binary_class_pred = (out > 0)#.float()
            binary_class_ground = (iden > 0.5)
            total_correct += (binary_class_pred == binary_class_ground).sum().item()
        

        test_accuracy = total_correct / (len(test_loader.dataset)*40)
        print("epoch", epoch, "test loss", total_loss / len(test_loader), " train loss", train_loss / len(train_loader), "test accuracy", test_accuracy)

In [ ]:
# save model
torch.save(model.state_dict(), "/mnt/rd/clip_to_features.pt")

In [ ]:
# lets look at test dataset
emb, iden = test_dataset[0]


In [ ]:
pred_iden = model(emb.unsqueeze(0).cuda().float()).squeeze()#.cpu().detach().numpy()

In [ ]:
import numpy as np
# Convert logits to probabilities using sigmoid function
probabilities = torch.sigmoid(pred_iden).cpu()

# Binarize probabilities with a threshold of 0.5
binary_predictions = (probabilities >= 0.5)#.astype(int)

In [ ]:
binary_predictions

In [ ]:
iden_binary = (torch.Tensor(iden) >= 0)#.astype(int)

In [ ]:
iden_binary

In [ ]:
accuracy = (binary_predictions == iden_binary).sum().item() / len(iden_binary)

In [ ]:
accuracy